# Ⅰ.数据导入+列名简化

In [2]:
import numpy as np
import pandas as pd
path1 = "C:/Users/Sinso/Desktop/1.xlsx"
path2 = "C:/Users/Sinso/Desktop/2.xlsx"
data1 = pd.read_excel(path1)
data2 = pd.read_excel(path2)
data1 = data1.rename(columns = {'法人单位名称':'单位名称'})
data2 = data2.rename(columns = {'全年工资报酬合计(元)':'全年工资'})
data2 = pd.merge(data2,data1,on='单位名称')
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5865 entries, 0 to 5864
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   单位名称                5865 non-null   object 
 1   学历                  5865 non-null   object 
 2   小类                  5536 non-null   object 
 3   中类                  5865 non-null   object 
 4   大类                  5865 non-null   object 
 5   管理岗位／专业技术职称／职业技能等级  5865 non-null   object 
 6   全年工资                5865 non-null   float64
 7   门类                  5865 non-null   object 
 8   企业规模                5865 non-null   object 
dtypes: float64(1), object(8)
memory usage: 458.2+ KB


# Ⅱ.数据预处理

有缺失值，但**只有列‘小类’具有缺失值**

In [3]:
datanan = data2[data2['小类'].isnull()]
datanan[datanan['中类'] == '其他专业技术人员'].shape == datanan.shape

True

发现含有缺失值的项对应的中类都是‘其他专业技术人员’，所以这些小类都用‘其他专业技术人员’进行填充.

In [4]:
data2['小类'] = data2['小类'].fillna('其他专业技术人员')

忽略企业员工人数小于10的数据，后续对企业评估减小误差

In [5]:
data3_2_1 = data2.groupby(by=['单位名称']).count()
data3 = data2.set_index('单位名称')
data3 = data3.drop(index=data3_2_1[data3_2_1['小类']<10].index)

# Ⅲ.数据分析

## 1.规模分析

In [6]:
data3_1_1 = data2.set_index(['企业规模'])
data3_1_2 = data2.groupby(by='企业规模').mean(numeric_only=True).sort_values(by='全年工资')
for each in data3_1_2.index:
    data3_1_2.loc[each,'员工人数'] = len(data3_1_1.loc[each])
data3_1_2 = data3_1_2.rename(columns={'全年工资':'全年平均工资'})
data3_1_2

,全年平均工资,员工人数
企业规模,,
小型,78553.745302,596.0
大型,93099.175563,2959.0
中型,97515.290104,2213.0
微型,139712.335670,97.0


**结论：** 微型企业人少，应证了一句话：高风险高回报，平均工资最高，微型企业可能也是创业集中点，在掌握好方向后可能创收比较大；中型企业与大型企业人数与工资水平相差不大，但小型企业与中大型相比全年工资低了一万多，所以从总方面来说，**建议去中大型企业，不建议小型与微型（需要做风险评估）**

## 2.企业分析

### 2.1 企业间学历竞争

**对企业分析的数据源采用经过删除小企业之后的数据 data3**

先给出评判比例：

研究生（含博士、硕士）：35% 

大学本科：28%

大学专科：20%

高中、中专或技校：12%

初中及以下：5%

In [7]:
data3_2_1 = data3.groupby(by=['单位名称','学历']).count()
data3_2_3 = pd.DataFrame(data3_2_1['小类'])
data3_2_3.rename(columns={'小类':'对应学历人数'},inplace=True)
data3_2_1 = data3.groupby(by=['单位名称']).count()
data3_2_4 = pd.DataFrame(data3_2_1['小类'])
data3_2_4.rename(columns={'小类':'总人数'},inplace=True)
data3_2_4.insert(1,'得分',np.zeros(data3_2_4.shape[0],np.int64))
scorepor = np.array([0.35,0.28,0.2,0.12,0.05])
cir = 0
for key1,key2 in data3_2_3.index:
    porpo = data3_2_3.loc[(key1,key2),'对应学历人数']/data3_2_4.loc[key1,'总人数']
    data3_2_4.loc[key1,'得分']+=100*porpo*scorepor[cir%5]
    cir+=1
data3_2_4.sort_values(ascending=False,by='得分',inplace=True)
data3_2_4.head(6)

,总人数,得分
单位名称,,
AC,42,35.000000
Z,33,33.242424
H,100,31.510000
AM,67,28.626866
N,12,26.333333
V,60,26.266667


企业AC，Z，H，AM，N，V学历较其他企业高

### 2.2 企业间工资水平比较

In [8]:
data3_2_6 = data3.groupby(by='单位名称').mean(numeric_only=True).sort_values(by='全年工资')
data3_2_6 = data3_2_6.rename(columns={'全年工资':'平均全年工资'})
data3_2_6.insert(1,'总人数',data3_2_4['总人数'])
data3_2_6.sort_values(by='全年平均工资',ascending=False,inplace=True)
data3_2_6.head(6)

,全年平均工资,总人数
单位名称,,
H,268267.338800,100
AH,211672.348929,140
Q,207234.429545,22
P,204540.458824,68
Z,188521.454848,33
K,159270.057143,35


企业H,AH,Q,P,Z,K平均工资高，尤其H公司以26万年高薪领跑

### 2.3 同行业选择就业企业倾向

In [68]:
data3_2_7 = data2.groupby(by=['大类']).count()
data3_2_8 = data2.groupby(by=['大类','单位名称']).count()
data3_2_9 = pd.DataFrame(data3_2_8['小类'])
data3_2_9.rename(columns={'小类':'总人数'},inplace=True)
length_dl = len(data3_2_9.index.levels[0])
for each in range(length_dl):
    data_ls = data3_2_9.loc[data3_2_9.index.levels[0][each]]
    data_ls.sort_values(by='总人数',ascending=False,inplace=True)
    total = data3_2_7.iloc[each,0]
    data_ls = data_ls.head(3)
    proportion1 = data_ls.iloc[0,0]/total
    proportion2 = data_ls.iloc[1,0]/total
    proportion3 = data_ls.iloc[2,0]/total
    proportion1 = "%.2f%%" % (proportion1 * 100)
    proportion2 = "%.2f%%" % (proportion2 * 100)
    proportion3 = "%.2f%%" % (proportion3 * 100)
    print('对于大类“',data3_2_9.index.levels[0][each],'”，去企业',data_ls.index[0],data_ls.index[1],\
          data_ls.index[2],'较多，占比分别为',proportion1,proportion2,proportion3)

对于大类“ 专业技术人员 ”，去企业 X D AA 较多，占比分别为 17.86% 11.94% 10.87%
对于大类“ 办事人员 ”，去企业 X P C 较多，占比分别为 23.68% 11.18% 7.57%
对于大类“ 单位负责人 ”，去企业 AH J D 较多，占比分别为 15.13% 14.42% 10.64%
对于大类“ 商业、服务业人员 ”，去企业 X G AE 较多，占比分别为 78.71% 3.20% 2.86%
对于大类“ 生产运输工人 ”，去企业 X J D 较多，占比分别为 35.01% 15.85% 12.91%


由于企业X是超大型企业，所以分析上很多行业都是去的X偏多，对于商品、服务业人员聚集最多。

In [132]:
data3_2_10 = data2.groupby(by=['小类']).count()
data3_2_11 = data2.groupby(by=['小类','单位名称']).count()
data3_2_12 = pd.DataFrame(data3_2_11['大类'])
data3_2_12.rename(columns={'大类':'总人数'},inplace=True)
length_dl = len(data3_2_12.index.levels[0])
data3_2_12.insert(1,'占比',np.nan)
for each in range(length_dl):
    index_ls = data3_2_12.index.levels[0][each] #小类的索引
    total = data3_2_10.loc[index_ls,'全年工资']
    data_ls = data3_2_12.loc[index_ls]
    if total > 10 & data_ls.shape[0] > 3:
        data_ls = data3_2_12.loc[data3_2_12.index.levels[0][each]].copy()
        data_ls.sort_values(by='总人数',ascending=False,inplace=True)
        total = data3_2_10.iloc[each,0]
        data_ls = data_ls.head(3)
        proportion1 = data_ls.iloc[0,0]/total
        proportion2 = data_ls.iloc[1,0]/total
        proportion3 = data_ls.iloc[2,0]/total
        proportion1 = "%.2f%%" % (proportion1 * 100)
        proportion2 = "%.2f%%" % (proportion2 * 100)
        proportion3 = "%.2f%%" % (proportion3 * 100)
        data3_2_12.loc[(index_ls,data_ls.index[0]),'占比'] = proportion1
        data3_2_12.loc[(index_ls,data_ls.index[1]),'占比'] = proportion2
        data3_2_12.loc[(index_ls,data_ls.index[2]),'占比'] = proportion3
        print('对于小类“',data3_2_12.index.levels[0][each],'”，总人数为',total,'，去企业',data_ls.index[0],data_ls.index[1],\
             data_ls.index[2],'较多，占比分别为',proportion1,proportion2,proportion3)
data3_2_12.dropna(inplace=True)
data3_2_12.sort_values(by='占比')
data3_2_12.sort_index()

对于小类“ 人事部门经理 ”，总人数为 16 ，去企业 AH AE K 较多，占比分别为 25.00% 12.50% 12.50%
对于小类“ 人力资源专业人员 ”，总人数为 27 ，去企业 H AA E 较多，占比分别为 25.93% 22.22% 14.81%
对于小类“ 企业总经理 ”，总人数为 55 ，去企业 X H M 较多，占比分别为 10.91% 9.09% 9.09%
对于小类“ 其他专业技术人员 ”，总人数为 329 ，去企业 X E AA 较多，占比分别为 55.93% 26.14% 5.47%
对于小类“ 其他企业中高级管理人员 ”，总人数为 165 ，去企业 J D E 较多，占比分别为 36.97% 23.64% 13.94%
对于小类“ 其他办事人员和有关人员 ”，总人数为 108 ，去企业 C R E 较多，占比分别为 41.67% 14.81% 13.89%
对于小类“ 其他职能部门经理 ”，总人数为 57 ，去企业 AH AE AA 较多，占比分别为 21.05% 15.79% 14.04%
对于小类“ 建筑工程技术人员 ”，总人数为 138 ，去企业 A T C 较多，占比分别为 50.00% 34.78% 5.07%
对于小类“ 生产经营部门经理 ”，总人数为 31 ，去企业 M AA AH 较多，占比分别为 29.03% 22.58% 19.35%
对于小类“ 行政业务办理人员 ”，总人数为 60 ，去企业 AD AA F 较多，占比分别为 20.00% 18.33% 18.33%
对于小类“ 行政事务处理人员 ”，总人数为 59 ，去企业 AA D O 较多，占比分别为 23.73% 23.73% 22.03%
对于小类“ 行政部门经理 ”，总人数为 21 ，去企业 F M AH 较多，占比分别为 28.57% 14.29% 9.52%
对于小类“ 财务部门经理 ”，总人数为 20 ，去企业 AH AE A 较多，占比分别为 20.00% 15.00% 10.00%
对于小类“ 销售和营销部门经理 ”，总人数为 31 ，去企业 AH AE AA 较多，占比分别为 32.26% 12.90% 9.68%


总人数      占比
小类          单位名称             
人事部门经理      AE      2  12.50%
            AH      4  25.00%
            K       2  12.50%
人力资源专业人员    AA      6  22.22%
            E       4  14.81%
            H       7  25.93%
企业总经理       H       5   9.09%
            M       5   9.09%
            X       6  10.91%
其他专业技术人员    AA     18   5.47%
            E      86  26.14%
            X     184  55.93%
其他企业中高级管理人员 D      39  23.64%
            E      23  13.94%
            J      61  36.97%
其他办事人员和有关人员 C      45  41.67%
            E      15  13.89%
            R      16  14.81%
其他职能部门经理    AA      8  14.04%
            AE      9  15.79%
            AH     12  21.05%
建筑工程技术人员    A      69  50.00%
            C       7   5.07%
            T      48  34.78%
生产经营部门经理    AA      7  22.58%
            AH      6  19.35%
            M       9  29.03%
行政业务办理人员    AA     11  18.33%
            AD     12  20.00%
            F      11  18.33%
行政事务处理人员    AA     14  23.73%
            D      14  23.73%
            O      13  22.03%
行政部门经理      AH      2   9.52%
            F       6  28.57%
            M       3  14.29%
财务部门经理      A       2  10.00%
            AE      3  15.00%
            AH      4  20.00%
销售和营销部门经理   AA      3   9.68%
            AE      4  12.90%
            AH     10  32.26%

## 3.工资水平分析

### 3.1 总体分析

In [141]:
gz_mean = data2['全年工资'].mean()
gz_max = data2['全年工资'].max()
gz_min = data2['全年工资'].min()
gz_median = data2['全年工资'].median()
print('这些员工 工资的中位数是',gz_median,'，平均工资为',gz_mean)

这些员工 工资的中位数是 83000.0 ，平均工资为 94058.2994458653


平均工资比中位数高一万多，这些员工工资差异较大

### 3.2 从学历分析

In [150]:
data3_3_1 = data2.groupby(by='学历').mean(numeric_only=True).sort_values(by='全年工资')
data3_3_2 = data3_3_1.rename(columns={'全年工资':'平均全年工资'})
data3_3_3 = data2.groupby(by='学历').count()
data3_3_2.insert(0,'总人数',data3_3_3['小类'])
data3_3_2

,总人数,平均全年工资
学历,,
高中、中专或技校,2195,76206.207804
初中及以下,1120,82668.927063
大学专科,1218,87659.881511
大学本科,1255,135555.735155
研究生（含博士、硕士）,77,193479.461169


学历越高平均工资越高，尤其研究生平均工资接近20万

### 3.3 从技能等级

In [152]:
data3_3_4 = data2.groupby(by='管理岗位／专业技术职称／职业技能等级').mean(numeric_only=True).sort_values(by='全年工资')
data3_3_5 = data3_3_4.rename(columns={'全年工资':'平均全年工资'})
data3_3_6 = data2.groupby(by='管理岗位／专业技术职称／职业技能等级').count()
data3_3_5.insert(0,'总人数',data3_3_6['小类'])
data3_3_5

,总人数,平均全年工资
管理岗位／专业技术职称／职业技能等级,,
没有取得专业技术职称,336,69083.765863
没有取得资格证书,3463,78277.633971
初级技能,51,78882.447451
管理类员工岗（其它管理岗）,360,80107.736722
高级技能,149,85913.291141
中级技能,58,86156.831379
初级职称,301,90077.171296
技师,66,91777.768636
中级职称,283,111301.007456
